In [1]:
import pymongo                                                                                                                                                                                                                               
from pymongo import MongoClient                                                                                                                                                                                                              
from pprint import pprint#, pformat, PrettyPrinter                                                                                                                                                                                            
# from bson.son import SON                                                                                                                                                                                                                     
from bson.objectid import ObjectId                                                                                                                                                                                                           
# from os.path import exists                                                                                                                                                                                                                   
# import json                                                                                                                                                                                                                                  
# from ipy_table import *        

from mongotools import *

In [2]:
def formatRow(row):
    formats = { 2: u"{}: {}", 3: u"{}: {} ({})" }
    row_format = formats.get( len(row), u"{} " * len(row) )
    return (row_format.format(*row.values()))

In [3]:
# Connect to database and create variables for the collections                                                                                                                                                                                 
client = MongoClient()                                                                                                                                                                                                                       
db = client.lobbyradar                                                                                                                                                                                                                       
entities, relations = db.entities, db.relations

print("%s Entities and %s Relations loaded." % (entities.count(), relations.count())) 

26380 Entities and 32137 Relations loaded.


In [4]:
etools = MongoHelper(entities)
rtools = MongoHelper(relations)

# Ontologie
Um eine Ontologie zu bauen müssen wir uns auf Klasse und Eigenschaften festlegen. Es geht nun also darum, die Daten gut zu untersuchen und die wichtigsten Klassen daraus abzuleiten.

Grundlegend geht es in diesem Datensatz um Vernetzung, weshalb die "Verbindung" so gut wie allen Relationen zu Grunde liegt.

Auf Seiten der Relationen sind die Felder
 - Tags
 - data.desc
 - data.format
 - __data.value.position__
 - data.value.type
 - __data.value.desc__
 - __data.value.amount__
 - __data.value.activity__

besonders aussagekräftig und bieten viele Daten.

### Tags
Die Tags bieten relativ viel Informationen, dafür leider unstrukturiert. Da es viele wenig genutzte Tags gibt, schränken wir die Suche etwas ein.

In [5]:
# Filter Tags with less than 100 occurences
pipeline = make_pipeline(unwind="tags", group="tags", sort=True, min_count=100)

tags = relations.aggregate(pipeline , cursor={})
printCols( [ formatRow(row) for row in tags ], spacing=20)

3926: partei        2552: nebentaetigkeit1804: committee     1151: pr-und-lobbyagentur1081: spende        
1081: parteispende  1047: anwaltskanzlei675: dax            629: mdb            603: laender        
557: kabinette      496: aufsichtsrat   466: pharma         281: seitenwechsler 245: bank           
230: regierung      229: kabinett       220: ruestung       192: vorstand       174: fernsehrat     
168: fernsehen      154: auto           


### data.XX.desc  | data.XX.format

In [6]:
desc_pipeline = make_pipeline(unwind="data", group="data.desc", sort=True, min_count=100)
format_pipeline = make_pipeline(unwind="data", group="data.format", sort=True, min_count=100)
desc = relations.aggregate(desc_pipeline, cursor={})
form = relations.aggregate(format_pipeline, cursor={})

printCols( [ u"{:<60} {:<50}".format(d, f) for (d, f) in zip(desc, form) ], columns=1 )

{u'count': 26258, u'_id': u'Verbindung'}                     {u'count': 26258, u'_id': u'association'}         
{u'count': 9673, u'_id': u'Parteispende'}                    {u'count': 9673, u'_id': u'donation'}             
{u'count': 2673, u'_id': u'Angaben zur Nebent\xe4tigkeit'}   {u'count': 2673, u'_id': u'activity'}             



Hier sehen wir deutlich, dass data.desc und data.format so gut wie deckungsgleich sehen. Es reicht also, wenn wir eines der Felder betrachten. 
Auch data.key ist fast gleichbedeutend damit.

### data.value.position

In [7]:
position_pipeline = make_pipeline(unwind="data", group="data.value.position", sort=True, min_count=100)
position = relations.aggregate(position_pipeline, cursor={})

printCols( [ formatRow(row) for row in position ], columns=4, spacing=40)

16054: Vorstand                         11394: None                             2635: Mitglied                          993: Ordentliches Mitglied              
838: Stellvertretendes Mitglied         837: Arbeitsverhältnis                  459: Aufsichtsratsmitglied              404: Mitglied des Kuratoriums           
231: Mitglied des Beirates              204: Mitglied des Aufsichtsrates        173: Mitglied des Vorstandes            161: Vorstandsmitglied                  
122: Staatssekretär                     115: Mitglied im Rundfunkrat            111: Mitglied des Kreistages            


Natürlicherweise sind alle Vorstände auch immer assoziiert mit dem jeweiligen Partner:

In [8]:
p = [
 {'$unwind': '$data'},
 {'$match': {'data.format': {'$eq': "association"}}},
 {'$group': {'_id': { 'position': '$data.value.position', 'format': '$data.format'}, 'count': {'$sum': 1}}},
 {'$sort': {'count': -1}},
 {'$match': {'count': {'$gt': 100}}} 
]
printCols( [ formatRow(row) for row in relations.aggregate(p) ], columns=2, spacing=90)

16054: {u'position': u'Vorstand', u'format': u'association'}                              2635: {u'position': u'Mitglied', u'format': u'association'}                               
1490: {u'format': u'association'}                                                         993: {u'position': u'Ordentliches Mitglied', u'format': u'association'}                   
838: {u'position': u'Stellvertretendes Mitglied', u'format': u'association'}              837: {u'position': u'Arbeitsverh\xe4ltnis', u'format': u'association'}                    
459: {u'position': u'Aufsichtsratsmitglied', u'format': u'association'}                   161: {u'position': u'Vorstandsmitglied', u'format': u'association'}                       
122: {u'position': u'Staatssekret\xe4r', u'format': u'association'}                       115: {u'position': u'Mitglied im Rundfunkrat', u'format': u'association'}                 



Dass eine Spende zusammen mit einer "Position" eintritt ist eher seltener..

In [9]:
p = [
 {'$unwind': '$data'},
 {'$match': {'data.format': {'$eq': "donation"}}},
 {'$group': {'_id': { 'position': '$data.value.position', 'format': '$data.format'}, 'count': {'$sum': 1}}},
 {'$sort': {'count': -1}},
 {'$match': {'count': {'$gt': 100}}} 
]
printCols( [ formatRow(row) for row in relations.aggregate(p) ], columns=2, spacing=90)

9673: {u'format': u'donation'}                                                            


Bei der activity gibt es wieder Personen, die eine Position innehaben.

In [10]:
p = [
 {'$unwind': '$data'},
 {'$match': {'data.format': {'$eq': "activity"}}},
 {'$group': {'_id': { 'position': '$data.value.position', 'format': '$data.format'}, 'count': {'$sum': 1}}},
 {'$sort': {'count': -1}},
 {'$match': {'count': {'$gt': 100}}} 
]
printCols( [ formatRow(row) for row in relations.aggregate(p) ], columns=2, spacing=90)

324: {u'position': u'Mitglied des Kuratoriums', u'format': u'activity'}                   229: {u'position': u'Mitglied des Beirates', u'format': u'activity'}                      
228: {u'position': None, u'format': u'activity'}                                          203: {u'position': u'Mitglied des Aufsichtsrates', u'format': u'activity'}                
169: {u'position': u'Mitglied des Vorstandes', u'format': u'activity'}                    110: {u'position': u'Mitglied des Kreistages', u'format': u'activity'}                    



### data.value.type

In [11]:
type_pipeline = make_pipeline(unwind="data", group="data.value.type", sort=True, min_count=50)
types = relations.aggregate(type_pipeline, cursor={})

printCols( [ formatRow(row) for row in types ], columns=2)

17212: executive              12338: None                   
5129: member                  1598: job                     
1445: business                633: government               
82: subsidiary                64:                           



In [12]:
subsidiary = relations.find({ "data.value.type": {"$eq": "subsidiary"} }, {'tags': 1, 'data.value': 1, 'entities': 1}).limit(3)
for r in subsidiary:
    pprint(r)

{u'_id': ObjectId('54ca5b4266c1658e2ecfbdb9'),
 u'data': [{u'value': {u'position': u'', u'type': u'subsidiary'}}],
 u'entities': [ObjectId('54c2a4cafe6a42c82bbab93d'),
               ObjectId('54bd3ca38b934da0634177da')],
 u'tags': [u'ruestung']}
{u'_id': ObjectId('54db26ae45666daa087dd7a7'),
 u'data': [{u'value': {u'position': u'', u'type': u'subsidiary'}}],
 u'entities': [ObjectId('54c2a4b3fe6a42c82bbaaf7e'),
               ObjectId('54bd3ca18b934da063417539')],
 u'tags': []}
{u'_id': ObjectId('54db280545666daa087dd7a9'),
 u'data': [{u'value': {u'position': u'', u'type': u'subsidiary'}}],
 u'entities': [ObjectId('54c2a4b3fe6a42c82bbaaf7e'),
               ObjectId('54cb842abbde97945a06a06f')],
 u'tags': []}


In [13]:
def lookup_relation(relation_id):
    relation = relations.find_one({ "_id": relation_id }, {'tags': 1, 'data': 1, 'entities': 1})  
    entity_ids = relation.get('entities')
    pprint(relation.get('data'))
    print(etools.lookup_entity(entitiy_ids[0], {'name': 1}))
    print(etools.lookup_entity(entitiy_ids[1], {'name': 1}))
    
def lookup_by_value_desc(value_desc):
    relation = relations.find_one({ "data.value.desc": value_desc }, {'tags': 1, 'data': 1, 'entities': 1})  
    entity_ids = relation.get('entities')
    pprint(relation.get('data'))
    print(etools.lookup_entity(entitiy_ids[0], {'name': 1}))
    print(etools.lookup_entity(entitiy_ids[1], {'name': 1}))

In [14]:
entitiy_ids = (ObjectId('54c2a4cafe6a42c82bbab93d'), ObjectId('54bd3ca38b934da0634177da'))
print(etools.lookup_entity(entitiy_ids[0], {'name': 1}))
print(etools.lookup_entity(entitiy_ids[1], {'name': 1}))

{u'_id': ObjectId('54c2a4cafe6a42c82bbab93d'), u'name': u'Jena Optronic GmbH'}
{u'_id': ObjectId('54bd3ca38b934da0634177da'), u'name': u'Airbus Group'}


In [15]:
entitiy_ids = (ObjectId('54c2a4b3fe6a42c82bbaaf7e'),
               ObjectId('54bd3ca18b934da063417539'))
print(etools.lookup_entity(entitiy_ids[0], {'name': 1}))
print(etools.lookup_entity(entitiy_ids[1], {'name': 1}))

{u'_id': ObjectId('54c2a4b3fe6a42c82bbaaf7e'), u'name': u'Bertelsmann SE & Co. KGaA'}
{u'_id': ObjectId('54bd3ca18b934da063417539'), u'name': u'Bertelsmann-Stiftung'}


In [16]:
entitiy_ids = (ObjectId('54c2a4b3fe6a42c82bbaaf7e'),
               ObjectId('54cb842abbde97945a06a06f'))
print(etools.lookup_entity(entitiy_ids[0], {'name': 1}))
print(etools.lookup_entity(entitiy_ids[1], {'name': 1}))

{u'_id': ObjectId('54c2a4b3fe6a42c82bbaaf7e'), u'name': u'Bertelsmann SE & Co. KGaA'}
{u'_id': ObjectId('54cb842abbde97945a06a06f'), u'name': u'RTL Television GmbH'}


### data.value.desc

In [17]:
desc_pipeline = make_pipeline(unwind="data", group="data.value.desc", sort=True, min_count=50)
descs = relations.aggregate(desc_pipeline, cursor={})

printCols( [ formatRow(row) for row in descs ], columns=2, spacing=70)

34325: None                                                           1435: Geschäftsverbindung                                             
1220: Funktionen in Vereinen, Verbänden und Stiftungen                682: Funktionen in Körperschaften und Anstalten des öffentlichen Rechts
468: Funktionen in Unternehmen                                        298: Entgeltliche Tätigkeiten neben dem Mandat                        
58: Bundestag Spenden von Personen und Beiträge von Mandatsträgern    


In [18]:
lookup_by_value_desc("Funktionen in Unternehmen")

[{u'auto': True,
  u'created': datetime.datetime(2015, 1, 30, 13, 16, 52, 128000),
  u'desc': u'Angaben zur Nebent\xe4tigkeit',
  u'format': u'activity',
  u'id': u'97748cdc9e054fcd94a1f98f398cfc84ee2a2762291348635b4a79384bf5339a',
  u'key': u'activity',
  u'updated': datetime.datetime(2015, 1, 30, 13, 16, 52, 128000),
  u'value': {u'activity': None,
             u'desc': u'Funktionen in Unternehmen',
             u'end': None,
             u'level': 0,
             u'periodical': u'einmalig',
             u'place': None,
             u'position': u'Mitglied des Aufsichtsrates',
             u'start': None,
             u'unsalaried': True}}]
{u'_id': ObjectId('54c2a4b3fe6a42c82bbaaf7e'), u'name': u'Bertelsmann SE & Co. KGaA'}
{u'_id': ObjectId('54cb842abbde97945a06a06f'), u'name': u'RTL Television GmbH'}


In [19]:
lookup_by_value_desc("Geschäftsverbindung")

[{u'desc': u'Verbindung',
  u'format': u'association',
  u'importer': u'created by pr importer',
  u'key': u'association',
  u'value': {u'desc': u'Gesch\xe4ftsverbindung',
             u'sources': [u'http://www.bohnen-kallmorgen.com/'],
             u'type': u'business'}}]
{u'_id': ObjectId('54c2a4b3fe6a42c82bbaaf7e'), u'name': u'Bertelsmann SE & Co. KGaA'}
{u'_id': ObjectId('54cb842abbde97945a06a06f'), u'name': u'RTL Television GmbH'}


In [20]:
lookup_by_value_desc("Funktionen in Vereinen, Verbänden und Stiftungen")

[{u'auto': True,
  u'created': datetime.datetime(2015, 1, 30, 13, 16, 53, 6000),
  u'desc': u'Angaben zur Nebent\xe4tigkeit',
  u'format': u'activity',
  u'id': u'bd5fff82726759130d968abda0494dd38ff13e2d66e432b18c6c8dab0961043f',
  u'key': u'activity',
  u'updated': datetime.datetime(2015, 1, 30, 13, 16, 53, 6000),
  u'value': {u'activity': None,
             u'desc': u'Funktionen in Vereinen, Verb\xe4nden und Stiftungen',
             u'end': None,
             u'level': 0,
             u'periodical': u'einmalig',
             u'place': None,
             u'position': u'Vorsitzende',
             u'start': None,
             u'unsalaried': True}},
 {u'desc': u'Verbindung',
  u'format': u'association',
  u'importer': u'created by lobbyliste importer',
  u'key': u'association',
  u'value': {u'position': u'Vorstand',
             u'sources': [u'http://bundestag.de/blob/189476/8989cc5f5f65426215d7e0233704b20a/lobbylisteaktuell-data.pdf'],
             u'type': u'executive'}}]
{u'_id': Obj

In [21]:
lookup_by_value_desc("Funktionen in Körperschaften und Anstalten des öffentlichen Rechts")

[{u'auto': True,
  u'created': datetime.datetime(2015, 1, 30, 13, 16, 51, 597000),
  u'desc': u'Angaben zur Nebent\xe4tigkeit',
  u'format': u'activity',
  u'id': u'd94f252e5e8a91dab2063c1db592f3c49f6f8db5ab68bf3617d415cb0eb87f23',
  u'key': u'activity',
  u'updated': datetime.datetime(2015, 1, 30, 13, 16, 51, 597000),
  u'value': {u'activity': None,
             u'desc': u'Funktionen in K\xf6rperschaften und Anstalten des \xf6ffentlichen Rechts',
             u'end': None,
             u'level': 0,
             u'periodical': u'einmalig',
             u'place': None,
             u'position': u'Mitglied des Stadtrates',
             u'start': None,
             u'unsalaried': False}}]
{u'_id': ObjectId('54c2a4b3fe6a42c82bbaaf7e'), u'name': u'Bertelsmann SE & Co. KGaA'}
{u'_id': ObjectId('54cb842abbde97945a06a06f'), u'name': u'RTL Television GmbH'}


In [22]:
lookup_by_value_desc("Bundestag Spenden von Personen und Beiträge von Mandatsträgern")

[{u'auto': True,
  u'created': datetime.datetime(2015, 1, 23, 19, 47, 42, 389000),
  u'desc': u'Parteispende',
  u'format': u'donation',
  u'id': u'be29110dcc2433e7fa8472b2a5632d28467f9ba7ea986dc9a7e712969172effa',
  u'importer': u'created by parteispenden importer',
  u'key': u'donation',
  u'updated': datetime.datetime(2015, 1, 23, 19, 47, 42, 389000),
  u'value': {u'amount': 10750,
             u'sources': [u'http://apps.opendatacity.de/parteispenden-recherche/assets/data/parteispenden.json'],
             u'year': 2006}},
 {u'auto': True,
  u'created': datetime.datetime(2015, 1, 23, 19, 47, 57, 433000),
  u'desc': u'Parteispende',
  u'format': u'donation',
  u'id': u'45c621a0ca5f94d85787b0ff32df43548181c8a98525b8b09177689c11c0e117',
  u'importer': u'created by parteispenden importer',
  u'key': u'donation',
  u'updated': datetime.datetime(2015, 1, 23, 19, 47, 57, 433000),
  u'value': {u'amount': 13911,
             u'sources': [u'http://apps.opendatacity.de/parteispenden-recherche/

### data.value.amount

In [23]:
amount_pipeline = make_pipeline(unwind="data", group="data.value.amount", sort=True, min_count=50)
amounts = relations.aggregate(amount_pipeline, cursor={})

printCols( [ formatRow(row) for row in amounts ], columns=2)

28934: None                   272: 15000                    
263: 20000                    145: 25000                    
124: 50000                    117: 12000                    
116: 12782.25                 98: 15338.7                   
94: 30000                     81: 11000                     
65: 12500                     64: 10500                     
62: 25564.5                   57: 13000                     



In [24]:
amounts = relations.find({ "data.value.amount": {"$exists": 1}}).limit(2)
pprint(list(amounts))

[{u'_id': ObjectId('54c2a4f0fe6a42c82bbabb54'),
  u'created': datetime.datetime(2015, 1, 23, 19, 45, 52, 563000),
  u'data': [{u'auto': True,
             u'created': datetime.datetime(2015, 1, 23, 19, 45, 52, 532000),
             u'desc': u'Parteispende',
             u'format': u'donation',
             u'id': u'8744c78b0d974e477285b33b874cbaddd1962043cea900f5c4f3094ebb590ccb',
             u'importer': u'created by parteispenden importer',
             u'key': u'donation',
             u'updated': datetime.datetime(2015, 1, 23, 19, 45, 52, 532000),
             u'value': {u'amount': 16872.57,
                        u'sources': [u'http://apps.opendatacity.de/parteispenden-recherche/assets/data/parteispenden.json'],
                        u'year': 1994}},
            {u'auto': True,
             u'created': datetime.datetime(2015, 1, 23, 19, 47, 9, 848000),
             u'desc': u'Parteispende',
             u'format': u'donation',
             u'id': u'8c5fe66733aba6f4143094f464a0

### Beide relation-IDs stammen von einer Parteispende-Beziehung

In [31]:
relations.find_one({"_id": ObjectId('54c2a4f0fe6a42c82bbabb54')})

{u'_id': ObjectId('54c2a4f0fe6a42c82bbabb54'),
 u'created': datetime.datetime(2015, 1, 23, 19, 45, 52, 563000),
 u'data': [{u'auto': True,
   u'created': datetime.datetime(2015, 1, 23, 19, 45, 52, 532000),
   u'desc': u'Parteispende',
   u'format': u'donation',
   u'id': u'8744c78b0d974e477285b33b874cbaddd1962043cea900f5c4f3094ebb590ccb',
   u'importer': u'created by parteispenden importer',
   u'key': u'donation',
   u'updated': datetime.datetime(2015, 1, 23, 19, 45, 52, 532000),
   u'value': {u'amount': 16872.57,
    u'sources': [u'http://apps.opendatacity.de/parteispenden-recherche/assets/data/parteispenden.json'],
    u'year': 1994}},
  {u'auto': True,
   u'created': datetime.datetime(2015, 1, 23, 19, 47, 9, 848000),
   u'desc': u'Parteispende',
   u'format': u'donation',
   u'id': u'8c5fe66733aba6f4143094f464a0ca759580be5b0775f3b9545ae648ffe3824f',
   u'importer': u'created by parteispenden importer',
   u'key': u'donation',
   u'updated': datetime.datetime(2015, 1, 23, 19, 47, 9,

In [29]:
pprint(lookup_relation(ObjectId('54c2a4f0fe6a42c82bbabb54')))

[{u'auto': True,
  u'created': datetime.datetime(2015, 1, 23, 19, 45, 52, 532000),
  u'desc': u'Parteispende',
  u'format': u'donation',
  u'id': u'8744c78b0d974e477285b33b874cbaddd1962043cea900f5c4f3094ebb590ccb',
  u'importer': u'created by parteispenden importer',
  u'key': u'donation',
  u'updated': datetime.datetime(2015, 1, 23, 19, 45, 52, 532000),
  u'value': {u'amount': 16872.57,
             u'sources': [u'http://apps.opendatacity.de/parteispenden-recherche/assets/data/parteispenden.json'],
             u'year': 1994}},
 {u'auto': True,
  u'created': datetime.datetime(2015, 1, 23, 19, 47, 9, 848000),
  u'desc': u'Parteispende',
  u'format': u'donation',
  u'id': u'8c5fe66733aba6f4143094f464a0ca759580be5b0775f3b9545ae648ffe3824f',
  u'importer': u'created by parteispenden importer',
  u'key': u'donation',
  u'updated': datetime.datetime(2015, 1, 23, 19, 47, 9, 848000),
  u'value': {u'amount': 12500,
             u'sources': [u'http://apps.opendatacity.de/parteispenden-recherche

In [30]:
pprint(lookup_relation(ObjectId('54c2a4f0fe6a42c82bbabb55')))

[{u'auto': True,
  u'created': datetime.datetime(2015, 1, 23, 19, 45, 52, 647000),
  u'desc': u'Parteispende',
  u'format': u'donation',
  u'id': u'bafefe61854ea83d496d5d12d7c2f8d27f2b7f8290a8590b3373a28bf84e015b',
  u'importer': u'Parteispenden',
  u'key': u'donation',
  u'updated': datetime.datetime(2015, 1, 23, 19, 45, 52, 647000),
  u'value': {u'amount': 15338.7,
             u'sources': [u'http://apps.opendatacity.de/parteispenden-recherche/assets/data/parteispenden.json'],
             u'year': 1994}},
 {u'auto': True,
  u'created': datetime.datetime(2015, 1, 23, 19, 47, 1, 194000),
  u'desc': u'Parteispende',
  u'format': u'donation',
  u'id': u'5980ff7509aae08ab740314bcb36708d981226759d75b702458011a381d7b0c7',
  u'importer': u'Parteispenden',
  u'key': u'donation',
  u'updated': datetime.datetime(2015, 1, 23, 19, 47, 1, 194000),
  u'value': {u'amount': 15000,
             u'sources': [u'http://apps.opendatacity.de/parteispenden-recherche/assets/data/parteispenden.json'],
      

In [26]:
amounts = relations.find({ "data.value.amount": {"$exists": 1}}, {'data.value': 1}).limit(2)
pprint(list(amounts))

[{u'_id': ObjectId('54c2a4f0fe6a42c82bbabb54'),
  u'data': [{u'value': {u'amount': 16872.57,
                        u'sources': [u'http://apps.opendatacity.de/parteispenden-recherche/assets/data/parteispenden.json'],
                        u'year': 1994}},
            {u'value': {u'amount': 12500,
                        u'sources': [u'http://apps.opendatacity.de/parteispenden-recherche/assets/data/parteispenden.json'],
                        u'year': 2004}}]},
 {u'_id': ObjectId('54c2a4f0fe6a42c82bbabb55'),
  u'data': [{u'value': {u'amount': 15338.7,
                        u'sources': [u'http://apps.opendatacity.de/parteispenden-recherche/assets/data/parteispenden.json'],
                        u'year': 1994}},
            {u'value': {u'amount': 15000,
                        u'sources': [u'http://apps.opendatacity.de/parteispenden-recherche/assets/data/parteispenden.json'],
                        u'year': 2003}},
            {u'value': {u'amount': 15000,
                       

### data.value.activity

In [27]:
activity_pipeline = make_pipeline(unwind="data", group="data.value.activity", sort=True)
activities = relations.aggregate(activity_pipeline, cursor={})

printCols( [ formatRow(row) for row in activities ], columns=3, spacing=70)

38418: None                                                           62: Rechtsanwalt                                                      33: Vortrag                                                           
8: Betriebsleiter                                                     7: Beratung                                                           4: Rechtsanwältin                                                     
3: Rechtsanwalt und Notar                                             3: Wissenschaftliche Beratung                                         3: Publizistische Tätigkeit                                           
2: Autorin                                                            2: Betreuerin                                                         2: Landwirt/Geschäftsführer                                           
2: Wirtschaftsprüfer/Steuerberater                                    2: Landwirt                                                           2: Berater, frei

In [28]:
pprint(list(relations.find({ "data.value.activity": "Vortrag" }).limit(3)))

[{u'_id': ObjectId('54cb8444bbde97945a06a67e'),
  u'created': datetime.datetime(2015, 1, 30, 13, 16, 52, 332000),
  u'data': [{u'auto': True,
             u'created': datetime.datetime(2015, 1, 30, 13, 16, 52, 331000),
             u'desc': u'Angaben zur Nebent\xe4tigkeit',
             u'format': u'activity',
             u'id': u'b42966dfdc88953b7a7e6b49198f7af669f4d9ff5979ddcf2c9823d9252231a3',
             u'key': u'activity',
             u'updated': datetime.datetime(2015, 1, 30, 13, 16, 52, 331000),
             u'value': {u'activity': u'Vortrag',
                        u'desc': u'Entgeltliche T\xe4tigkeiten neben dem Mandat',
                        u'end': {u'year': 2013},
                        u'level': 2,
                        u'periodical': u'einmalig',
                        u'place': None,
                        u'position': None,
                        u'start': {u'year': 2013},
                        u'unsalaried': False}}],
  u'entities': [ObjectId('54c71e9234

In [33]:
list(relations.find().limit(3))

[{u'_id': ObjectId('54bd3c968b934da063413717'),
  u'created': datetime.datetime(2015, 1, 19, 17, 19, 18, 521000),
  u'data': [{u'desc': u'Verbindung',
    u'format': u'association',
    u'importer': u'created by lobbyliste importer',
    u'key': u'association',
    u'value': {u'position': u'Vorstand',
     u'sources': [u'http://bundestag.de/blob/189476/8989cc5f5f65426215d7e0233704b20a/lobbylisteaktuell-data.pdf'],
     u'type': u'executive'}}],
  u'entities': [ObjectId('54bd3c768b934da06340f4c5'),
   ObjectId('54bd3c768b934da06340f4c7')],
  u'importer': u'lobbyliste',
  u'tags': [],
  u'type': u'general',
  u'updated': datetime.datetime(2015, 1, 19, 17, 19, 18, 521000),
  u'weight': 1},
 {u'_id': ObjectId('54bd3c968b934da063413719'),
  u'created': datetime.datetime(2015, 1, 19, 17, 19, 18, 525000),
  u'data': [{u'desc': u'Verbindung',
    u'format': u'association',
    u'importer': u'created by lobbyliste importer',
    u'key': u'association',
    u'value': {u'position': u'Vorstand',
 